# Cliquet
Price a Cliquet using different models. Cliquets are frequently used as hedges for annuities with caps and floors.
For a definition of the payoff, and the dataclass to construct a cliquet timetable, see [qablet_contracts.eq.cliquet.Accumulator](https://qablet.github.io/qablet-contracts/examples/equity_cliquet)

In [1]:
import pandas as pd
from qablet.black_scholes.mc import LVMCModel
from qablet.heston.mc import HestonMCModel
from qablet_contracts.eq.cliquet import Accumulator

from data.spx_2005_09_15 import (
    heston_data,
    localvol_data,
    rbergomi_data,
    basic_info,
)
from src.model.rbergomi import rBergomiMCModel

### Contract
Create a Cliquet Contract fixing bi-monthly.

In [2]:
info = basic_info()
prc_dt = info["prc_dt"]
exp_dt = prc_dt + pd.DateOffset(years=1)

fix_dates = pd.bdate_range(prc_dt, exp_dt, freq="2BME")
global_floor = 0.0
local_floor = -0.03
local_cap = 0.05
timetable = Accumulator(
    "USD",
    "SPX",
    fix_dates,
    global_floor,
    local_floor,
    local_cap,
    state={"S_PREV": 1.0},
).timetable()

print(timetable["events"].to_pandas())

  track                      time   op  quantity     unit
0   NaN 2005-09-30 00:00:00+00:00  NaN       0.0     INIT
1   NaN 2005-11-30 00:00:00+00:00  NaN       0.0  CALCFIX
2   NaN 2006-01-31 00:00:00+00:00  NaN       0.0  CALCFIX
3   NaN 2006-03-31 00:00:00+00:00  NaN       0.0  CALCFIX
4   NaN 2006-05-31 00:00:00+00:00  NaN       0.0  CALCFIX
5   NaN 2006-07-31 00:00:00+00:00  NaN       0.0  CALCFIX
6       2006-07-31 00:00:00+00:00    >       0.0      USD
7       2006-07-31 00:00:00+00:00    +       1.0      ACC


### Compare Models
Compare different models, starting with Local Volatility Model.

In [3]:
model = LVMCModel()
price, _ = model.price(timetable, localvol_data())
print(f"Localvol price: {price}")

Localvol price: 0.08365943802285157


Heston Model

In [4]:
model = HestonMCModel()
price, _ = model.price(timetable, heston_data())
print(f"Heston price: {price}")

Heston price: 0.08584398141062825


Bergomi Model

In [5]:
model = rBergomiMCModel()
dataset = rbergomi_data()
dataset["MC"]["PATHS"] = 20_000  # very slow with 100_000, 1/250
dataset["MC"]["TIMESTEP"] = 1 / 100
price, _ = model.price(timetable, dataset)
print(f"rBergomi price: {price}")

rBergomi price: 0.08863768453233743
